In [ ]:
1.	import csv
2.	from langchain_core.documents import Document
3.	from langchain_openai import OpenAIEmbeddings, ChatOpenAI
4.	from langchain_community.vectorstores import FAISS
5.	from langchain.text_splitter import RecursiveCharacterTextSplitter
6.	from langchain.chains import RetrievalQA
7.
8.	# Step 1: Load CSV file
9.	csv_path = "customer_support_tickets.csv"
10.	documents = []
11.
12.	with open(csv_path, newline='', encoding='utf-8') as csvfile:
13.	   reader = csv.DictReader(csvfile)
14.	   for index, row in enumerate(reader):
15.	       # Build the content from subject, description, and resolution
16.	       content = f"""Customer Issue: {row['Ticket Subject']}\nDescription: {row['Ticket Description']}\nSupport Response: {row['Resolution']}"""
17.	       documents.append(Document(page_content=content, metadata={"source": f"ticket_{index}"}))
18.
19.	# Step 2: Chunking
20.	splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
21.	chunks = splitter.split_documents(documents)
22.
23.	# Step 3: Generate Embeddings
24.	embeddings = OpenAIEmbeddings(openai_api_key="API-KEY")
25.	vectorstore = FAISS.from_documents(chunks, embeddings)
26.	retriever = vectorstore.as_retriever()
27.
28.	# Step 4: Use an LLM model
29.	llm = ChatOpenAI(model_name="gpt-4o", temperature=0.2, openai_api_key="API-KEY")
30.
31.	# Step 5: Use a RAG chain
32.	rag_chain = RetrievalQA.from_chain_type(
33.	   llm=llm,
34.	   chain_type="stuff",
35.	   retriever=retriever,
36.	   return_source_documents=True
37.	)
38.
39.	# Step 6: Ask a query
40.	query = "What should I do if my software crashes?"
41.	result = rag_chain.invoke(query)
42.
43.	print("\nAnswer:\n", result['result'])
44.	print("\nSources:")
45.	for doc in result['source_documents']:


In [ ]:
1.	import csv
2.	from langchain.chains import LLMChain
3.	from langchain.prompts import PromptTemplate
4.	from langchain_openai import ChatOpenAI
5.
6.	# Step 1: load the dataset (reviews_dataset.csv)
7.	csv_path = "reviews_dataset.csv"
8.	comments = []
9.
10.	with open(csv_path, newline='', encoding='cp1252') as csvfile:
11.	   reader = csv.DictReader(csvfile)
12.	   for row in reader:
13.	       if row.get("Comments"):
14.	           comments.append(row["Comments"])
15.
16.	# Step 2: set up the LLM
17.	llm = ChatOpenAI(model_name="gpt-4o", temperature=0.2, openai_api_key="api key")
18.
19.	# Step 3: define a prompt that extracts polarity
20.	polarity_prompt = PromptTemplate(
21.	   input_variables=["review"],
22.	   template="""
23.	   Analyze the sentiment polarity (Positive, Neutral, or Negative) of the following customer review:
24.	   Review: {review}
25.	   """
26.	)
27.
28.	# Step 3: define a prompt that extracts aspect-based sentiment (product, service, delivery)
29.	aspect_prompt = PromptTemplate(
30.	   input_variables=["review"],
31.	   template="""
32.	   Analyze this customer review and extract sentiment for the following aspects: Product, Service, and Delivery.
33.	   Return the aspect sentiments in this format:
34.	   Product: [Positive/Neutral/Negative]
35.	   Service: [Positive/Neutral/Negative]
36.	   Delivery: [Positive/Neutral/Negative]
37.
38.	   Review: {review}
39.	   """
40.	)
41.
42.	# Step 4: build the prompt chains
43.	polarity_chain = polarity_prompt | llm
44.	aspect_chain = aspect_prompt | llm
45.
46.	# Step 5: run the analysis, and substitute the review argument
47.	for index, comment in enumerate(comments, 1):
48.	   print(f"\n--- Review #{index} ---")
49.	   print("Original Comment:", comment)
50.
51.	   polarity = polarity_chain.invoke({"review": comment}).content.strip()
52.	   aspects = aspect_chain.invoke({"review": comment}).content.strip()
53.
54.	   print("Polarity:", polarity)
55.	   print("Aspect-Based Sentiment:\n", aspects)


In [ ]:
1.	from langchain_core.documents import Document
2.	from langchain.text_splitter import RecursiveCharacterTextSplitter
3.	from langchain_community.vectorstores import FAISS
4.	from langchain_openai import OpenAIEmbeddings, ChatOpenAI
5.	from langchain.chains import RetrievalQA
6.
7.	# Step 1: Load the policy document
8.	with open("policy.txt", "r", encoding="utf-8") as f:
9.	   policy_text = f.read()
10.
11.	documents = [Document(page_content=policy_text)]
12.
13.	# Step 2: Chunk the policy document and create embeddings.
14.	splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
15.	chunks = splitter.split_documents(documents)
16.
17.	# Step 3: create the vector index
18.	embeddings = OpenAIEmbeddings(openai_api_key="api-key")
19.	vectorstore = FAISS.from_documents(chunks, embeddings)
20.	retriever = vectorstore.as_retriever()
21.
22.	# Step 4: set up the LLM
23.	llm = ChatOpenAI(model_name="gpt-4o", temperature=0.3, openai_api_key="api-key")
24.
25.	# Step 5: Set up the RAG system
26.	rag_chain = RetrievalQA.from_chain_type(
27.	   llm=llm,
28.	   chain_type="stuff",  # simple concatenation of retrieved policy text
29.	   retriever=retriever,
30.	   return_source_documents=True
31.	)
32.
33.	# Step 6: Show an example of handling a complaint
34.	complaint = "My sushi arrived cold and 45 minutes late. Can I get a refund?"
35.
36.	result = rag_chain.invoke(complaint)
37.
38.	# Step 7: Show resolution and evidence
39.	print("\nResolution:\n", result["result"])
40.	print("\nPolicy Evidence Used:")
41.	for doc in result['source_documents']:
42.	  print(f"- {doc.page_content[:150]}...")

In [ ]:
# Code 6.X. Proactive support trigger for a delivery platform (early signals + LLM messaging)

import csv
from collections import defaultdict
from datetime import datetime

from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate


# -----------------------------
# Step 1: Load operational events (e.g., delivery delays)
# events.csv columns:
# timestamp, zone, delay_minutes, reason
# -----------------------------
events_path = "events.csv"
zone_delay = {}  # latest delay per zone

with open(events_path, newline="", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    for row in reader:
        zone = row["zone"].strip()
        delay = int(row["delay_minutes"])
        ts = row["timestamp"].strip()
        # keep the latest delay snapshot per zone
        if zone not in zone_delay or ts > zone_delay[zone]["timestamp"]:
            zone_delay[zone] = {"timestamp": ts, "delay_minutes": delay, "reason": row["reason"].strip()}


# -----------------------------
# Step 2: Load interaction signals (early warning cues)
# interactions.csv columns:
# timestamp, customer_id, zone, channel, text
# channel examples: search, chat
# -----------------------------
interactions_path = "interactions.csv"
customer_signals = defaultdict(list)

with open(interactions_path, newline="", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    for row in reader:
        customer_id = row["customer_id"].strip()
        zone = row["zone"].strip()
        customer_signals[customer_id].append(
            {
                "timestamp": row["timestamp"].strip(),
                "zone": zone,
                "channel": row["channel"].strip(),
                "text": row["text"].strip(),
            }
        )


# -----------------------------
# Step 3: Simple heuristic pre-filter (cheap, non-LLM)
# We only send LLM work for customers in zones with active delays,
# and whose language suggests uncertainty or repeated checking.
# -----------------------------
RISK_PHRASES = [
    "where is my order",
    "still waiting",
    "late",
    "delay",
    "refund",
    "cancel",
    "what's happening",
    "no update",
    "taking so long",
    "again",
    "third time",
    "not responding",
]

def heuristic_risk_score(signals: list[dict]) -> int:
    score = 0
    # more interactions in short time -> higher risk
    score += min(len(signals), 5)
    # keywords -> higher risk
    for s in signals:
        t = s["text"].lower()
        if any(p in t for p in RISK_PHRASES):
            score += 2
        # searches are “early” signals (often precede complaints)
        if s["channel"].lower() == "search":
            score += 1
    return score


# -----------------------------
# Step 4: Set up the LLM (used only for (a) risk classification and (b) message generation)
# -----------------------------
llm = ChatOpenAI(model_name="gpt-4o", temperature=0.2, openai_api_key="API-KEY")

risk_prompt = PromptTemplate(
    input_variables=["delay_minutes", "reason", "signals"],
    template="""
You are a customer support analyst for a food delivery platform.
Given operational delay context and customer interaction signals, classify the customer's risk of complaining in the next hour.

Delay context:
- delay_minutes: {delay_minutes}
- reason: {reason}

Customer signals (latest first):
{signals}

Return ONLY one of: LOW, MEDIUM, HIGH
"""
)

message_prompt = PromptTemplate(
    input_variables=["delay_minutes", "reason"],
    template="""
You are a helpful customer support assistant for a food delivery platform.
Write a proactive message to a customer affected by a delivery delay.

Constraints:
- Be brief (2–4 sentences).
- Acknowledge the delay without blaming the customer.
- Provide a realistic expectation (mention the delay_minutes).
- Offer a helpful next step (e.g., tracking, support contact).
- Tone: calm, respectful, empathetic.

Delay minutes: {delay_minutes}
Reason: {reason}
"""
)

risk_chain = risk_prompt | llm
message_chain = message_prompt | llm


# -----------------------------
# Step 5: Run proactive detection + generate messages
# -----------------------------
PROACTIVE_THRESHOLD = 7  # heuristic threshold to consider running LLM classification

proactive_actions = []

for customer_id, signals in customer_signals.items():
    # sort signals newest first
    signals_sorted = sorted(signals, key=lambda x: x["timestamp"], reverse=True)
    zone = signals_sorted[0]["zone"]

    # only consider customers in zones with a known active delay snapshot
    if zone not in zone_delay:
        continue

    delay_minutes = zone_delay[zone]["delay_minutes"]
    reason = zone_delay[zone]["reason"]

    # cheap heuristic filter first
    h_score = heuristic_risk_score(signals_sorted)
    if h_score < PROACTIVE_THRESHOLD:
        continue

    # format signals for the LLM
    signals_text = "\n".join(
        [f"- [{s['timestamp']}] ({s['channel']}) {s['text']}" for s in signals_sorted[:8]]
    )

    # LLM risk classification
    risk_level = risk_chain.invoke(
        {
            "delay_minutes": delay_minutes,
            "reason": reason,
            "signals": signals_text,
        }
    ).content.strip().upper()

    # only send proactive messages for medium/high risk
    if risk_level in {"MEDIUM", "HIGH"}:
        proactive_message = message_chain.invoke(
            {
                "delay_minutes": delay_minutes,
                "reason": reason,
            }
        ).content.strip()

        proactive_actions.append(
            {
                "customer_id": customer_id,
                "zone": zone,
                "delay_minutes": delay_minutes,
                "risk_level": risk_level,
                "message": proactive_message,
            }
        )


# -----------------------------
# Step 6: Output (in production, you would dispatch via SMS/push/in-app message)
# -----------------------------
print("\n=== Proactive Support Actions ===")
for a in proactive_actions:
    print(f"\nCustomer: {a['customer_id']} | Zone: {a['zone']} | Delay: {a['delay_minutes']} min | Risk: {a['risk_level']}")
    print("Proactive Message:")
    print(a["message"])